In [4]:
import matplotlib.pyplot as plt
import numpy as np
import os
from matplotlib.ticker import ScalarFormatter, MaxNLocator
from scipy.special import eval_genlaguerre

# Простой прогресс бар без tqdm

def tqdm(iterable, total=None, desc=""):
    """Простая замена tqdm для случаев, когда библиотека недоступна"""
    if total is None:
        total = len(iterable)
    for i, item in enumerate(iterable):
        if i % max(1, total // 10) == 0 or i == total - 1:
            print(f"{desc}: {i+1}/{total} ({100*(i+1)/total:.1f}%)")
        yield item

# Геометрия треугольной апертуры

def triangle_vertices_cartesian(L):
    """Вершины равностороннего треугольника со стороной L"""
    R = L / np.sqrt(3)
    return np.array([
        [R * np.cos(0), R * np.sin(0)],
        [R * np.cos(2*np.pi/3), R * np.sin(2*np.pi/3)],
        [R * np.cos(4*np.pi/3), R * np.sin(4*np.pi/3)]
    ])

# Быстрая функция для Лагерр–Гауссова поля с векторной маской

def compute_lg_diffraction_fast(vertices, X, Y, z, k, k_z, m, n, w0, zR, N_int=100):
    # 1) Сетка апертуры
    x_min, y_min = np.min(vertices, axis=0)
    x_max, y_max = np.max(vertices, axis=0)
    xs = np.linspace(x_min, x_max, N_int)
    ys = np.linspace(y_min, y_max, N_int)
    x_ap, y_ap = np.meshgrid(xs, ys, indexing='ij')

    # 2) Векторная маска внутри треугольника
    v1, v2, v3 = vertices
    cross1 = (v2[0]-v1[0])*(y_ap-v1[1]) - (v2[1]-v1[1])*(x_ap-v1[0])
    cross2 = (v3[0]-v2[0])*(y_ap-v2[1]) - (v3[1]-v2[1])*(x_ap-v2[0])
    cross3 = (v1[0]-v3[0])*(y_ap-v3[1]) - (v1[1]-v3[1])*(x_ap-v3[0])
    mask = ((cross1>=0)&(cross2>=0)&(cross3>=0)) | ((cross1<=0)&(cross2<=0)&(cross3<=0))

    x_p = x_ap[mask]
    y_p = y_ap[mask]

    # Шаги интегрирования
    dx = (x_max - x_min) / (N_int - 1)
    dy = (y_max - y_min) / (N_int - 1)
    area_weight = dx * dy

    # 3) Пред-вычисления поперечного профиля LG на апертуре
    rho_p = np.hypot(x_p, y_p)
    # Угловая часть e^{i m phi} в декартовой форме
    if m >= 0:
        angular = (x_p + 1j*y_p)**m / (rho_p**m if m != 0 else 1)
    else:
        angular = (x_p - 1j*y_p)**abs(m) / (rho_p**abs(m))

    radial = (rho_p / w0)**abs(m) * eval_genlaguerre(n, abs(m), 2 * rho_p**2 / w0**2)
    gauss = np.exp(-rho_p**2 / w0**2)

    Psi0 = radial * angular * gauss * area_weight

    # Коэффициент градиентного слагаемого (эффективный)
    grad_coeff = 1j * (k * rho_p**2 / (2 * zR**2) + k_z - (2*n + abs(m) + 1) / zR)

    # 4) Интегрирование по точкам апертуры
    psi = np.zeros_like(X, dtype=complex)
    fourpi = 4 * np.pi
    for idx in range(x_p.size):
        xp, yp = x_p[idx], y_p[idx]
        psi0 = Psi0[idx]
        grad0 = grad_coeff[idx] * psi0

        R = np.sqrt((X - xp)**2 + (Y - yp)**2 + z**2)
        R = np.where(R < 1e-10, 1e-10, R)
        K = np.exp(1j * k * R) / (fourpi * R)

        surf = psi0 * (1j * k - 1/R) * (z / R)
        psi += K * (surf + grad0)

    return psi

# Параметры расчета
me = 0.511e6          # масса электрона (эВ)
E_k = 100e3           # кинетическая энергия (эВ)
E = 1 + E_k / me      # полная энергия (без использования далее, оставлено для контекста)
k = np.sqrt(2 * E_k / me)
lambda_e = (2 * np.pi) / k

theta = 1e-5          # малый угол раствора (рад)
k_z = k * np.cos(theta)

# Параметры LG
m_lg = 2              # азимутальный индекс (ℓ)
n_lg = 0              # радиальный индекс (p)
w0 = 2e5              # радиус пучка на апертуре (в тех же ед.)
zR = 0.5 * k * w0**2  # оценка зРэлея

# Треугольная апертура и область визуализации
L = 1e6
D = L / np.sqrt(3)
z_fraunhofer = D**2 / lambda_e

# Папка и параметры вывода
output_dir = "lg_triangle_zones_small_fonts"
os.makedirs(output_dir, exist_ok=True)
dpi = 300
img_format = "png"

# Sweep по z (логарифмически)
z_ref = z_fraunhofer
z_min_factor, z_max_factor = 0.05, 8.0
n_frames = 60
z_factors = np.geomspace(z_min_factor, z_max_factor, n_frames)
z_values = z_factors * z_fraunhofer

# Сетка наблюдения и интеграция
N_xy = 100
N_int = 100

# Масштаб области обзора (одинаковый угловой FOV)
b_ref = 1e6
gamma = 2.0  # компенсация I ∝ 1/z^2

# Геометрия апертуры
vertices = triangle_vertices_cartesian(L)

# 1) Рендерим кадры в память, фиксируя общий цветовой масштаб
print("Вычисление кадров...")
frames = []
for idx, z in tqdm(enumerate(z_values), total=len(z_values), desc="Вычисление"):
    b = b_ref * (z / z_ref)
    x = np.linspace(-b, b, N_xy)
    y = np.linspace(-b, b, N_xy)
    X, Y = np.meshgrid(x, y)

    psi = compute_lg_diffraction_fast(vertices, X, Y, z, k, k_z, m_lg, n_lg, w0, zR, N_int=N_int)
    I = np.abs(psi)**2
    I = I * (z / z_ref)**gamma
    frames.append((I, b, z))

# Общий робастный максимум для стабильной цветовой шкалы
vmax = np.percentile(np.concatenate([I.ravel() for I, _, _ in frames]), 99.5)

# 2) Сохраняем кадры с уменьшенными шрифтами
print("Сохранение кадров...")
single_figsize = (8, 6)
for idx, (I, b, z) in tqdm(enumerate(frames), total=len(frames), desc="Сохранение"):
    fig, ax = plt.subplots(figsize=single_figsize)
    im = ax.imshow(I, extent=[-b, b, -b, b], cmap="viridis", origin="lower", vmin=0, vmax=vmax)

    # Уменьшенные размеры шрифтов
    ax.set_title(f"LG m={m_lg}, n={n_lg}: z = {z*386e-15:.2e} m", fontsize=16)
    ax.set_xlabel("X position (pm)", fontsize=14)
    ax.set_ylabel("Y position (pm)", fontsize=14)
    ax.tick_params(axis="both", which="major", labelsize=12)
    ax.xaxis.set_major_locator(MaxNLocator(5))
    ax.yaxis.set_major_locator(MaxNLocator(5))
    ax.xaxis.get_offset_text().set_fontsize(12)
    ax.yaxis.get_offset_text().set_fontsize(12)

    cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label("Intensity", fontsize=14)
    cbar.ax.tick_params(labelsize=12)
    cbar.formatter = ScalarFormatter(useMathText=True)
    cbar.formatter.set_powerlimits((-3, 3))
    cbar.formatter.set_scientific(True)
    cbar.formatter.set_useOffset(False)
    cbar.update_normal(im)

    plt.tight_layout()
    filename = f"frame_{idx:04d}.{img_format}"
    filepath = os.path.join(output_dir, filename)
    plt.savefig(filepath, dpi=dpi, bbox_inches="tight", pad_inches=0.1)
    plt.close()

print(f"Сохранено {len(frames)} кадров в '{output_dir}'")
print("Все шрифты уменьшены:")
print("- Заголовок: 25 → 16")
print("- Подписи осей: 25 → 14")
print("- Цифры на осях: 20 → 12")
print("- Подписи цветовой шкалы: 25 → 14")
print("- Цифры на цветовой шкале: 20 → 12")


Вычисление кадров...
Вычисление: 1/60 (1.7%)
Вычисление: 7/60 (11.7%)
Вычисление: 13/60 (21.7%)
Вычисление: 19/60 (31.7%)
Вычисление: 25/60 (41.7%)
Вычисление: 31/60 (51.7%)
Вычисление: 37/60 (61.7%)
Вычисление: 43/60 (71.7%)
Вычисление: 49/60 (81.7%)
Вычисление: 55/60 (91.7%)
Вычисление: 60/60 (100.0%)
Сохранение кадров...
Сохранение: 1/60 (1.7%)
Сохранение: 7/60 (11.7%)
Сохранение: 13/60 (21.7%)
Сохранение: 19/60 (31.7%)
Сохранение: 25/60 (41.7%)
Сохранение: 31/60 (51.7%)
Сохранение: 37/60 (61.7%)
Сохранение: 43/60 (71.7%)
Сохранение: 49/60 (81.7%)
Сохранение: 55/60 (91.7%)
Сохранение: 60/60 (100.0%)
Сохранено 60 кадров в 'lg_triangle_zones_small_fonts'
Все шрифты уменьшены:
- Заголовок: 25 → 16
- Подписи осей: 25 → 14
- Цифры на осях: 20 → 12
- Подписи цветовой шкалы: 25 → 14
- Цифры на цветовой шкале: 20 → 12


In [ ]:
# Собрать анимацию 4 fps из сохранённых кадров
import glob
import imageio.v2 as imageio

img_dir = "lg_triangle_zones_small_fonts"  # при необходимости поменяйте на вашу папку
pattern = os.path.join(img_dir, "frame_*.png")
paths = sorted(glob.glob(pattern))

if not paths:
    raise FileNotFoundError(f"Не найдены кадры по шаблону: {pattern}")

print(f"Найдено кадров: {len(paths)}")

# Читаем изображения
images = [imageio.imread(p) for p in tqdm(paths, total=len(paths), desc="Чтение кадров")]

# GIF 4 fps
out_gif = os.path.join(img_dir, "animation_4fps.gif")
imageio.mimsave(out_gif, images, duration=0.25, loop=0)
print(f"GIF сохранён: {out_gif}")

# Попробуем также MP4 4 fps (если доступен ffmpeg)
out_mp4 = os.path.join(img_dir, "animation_4fps.mp4")
try:
    writer = imageio.get_writer(out_mp4, fps=4, codec="libx264", quality=8)
    for im in tqdm(images, total=len(images), desc="Запись MP4"):
        writer.append_data(im)
    writer.close()
    print(f"MP4 сохранён: {out_mp4}")
except Exception as e:
    print(f"MP4 пропущен (нет ffmpeg или кодека): {e}")
